In [1]:
import numpy as np
import matplotlib.pyplot as plt

from time import time
from IPython.display import clear_output

# Урок 3. Классификация. Логистическая регрессия

In [2]:
def calc_std_feat(x):
    res = (x - x.mean()) / x.std()
    return res

In [3]:
X = np.array([[1, 1, 1, 1, 1, 1, 1, 1, 1, 1],
              [1, 1, 2, 5, 3, 0, 5, 10, 1, 2],
              [500, 700, 750, 600, 1450, 800, 1500, 2000, 450, 1000],
              [1, 1, 2, 1, 2,  1, 3, 3, 1, 2]], dtype = np.float64)
y = np.array([0, 0, 1, 0, 1, 0, 1, 0, 1, 1], dtype = np.float64)

X_st = X.copy()
X_st[2, :] = calc_std_feat(X[2, :])

### 1*. Измените функцию calc_logloss так, чтобы нули по возможности не попадали в np.log.

#### Чтобы попал 0, y_pred должен равняться либо 0, либо 1. Т. е. в функции sigmoid np.exp должна возвращать либо 0, либо бесконечность. Второе маловероятно. 
#### Для борьбы с первым возникла идея изменить np.log(1.0 - y_pred) -> np.log(1.1 - y_pred). Или можно использовать ещё более меньшее число, например 1 + 1e-8. Это должно решить проблему, так как sigmoid не может вернуть число больше 1.

In [4]:
def calc_logloss(y, y_pred):
    err = - np.mean(y * np.log(y_pred) + (1.0 - y) * np.log(1.0 - y_pred))
    err = np.sum(err)
    return err

### 2. Подберите аргументы функции eval_model для логистической регрессии таким образом, чтобы log loss был минимальным.

In [5]:
def sigmoid(z):
    res = 1 / (1 + np.exp(-z))
    return res

In [6]:
def eval_model(X, y, iterations, alpha):
    np.random.seed(42)
    W = np.random.randn(X.shape[0])
    n = X.shape[1]
    for i in range(1, iterations + 1):
        z = np.dot(W, X)
        y_pred = sigmoid(z)
        err = calc_logloss(y, y_pred)
        W -= alpha * (1 / n * np.dot((y_pred - y), X.T))
        
    return W, err

In [7]:
iter_alpha_err_time = []

for i in range(1000, 20000, 100):        
    for alpha in [10 ** d for d in range(-10, 0, 1)]:
        start = time()
        W, err = eval_model(X_st, y, i, alpha)  
        time_interval = time() - start
        
        iter_alpha_err_time.append([i, alpha, err, time_interval])
        
        clear_output(wait=True)
        
        print('iteration\talpha\terror\t\t\ttime')
        print(f'{i}\t\t{alpha}\t{err}\t{time_interval}')
        

iteration	alpha	error			time
19900		0.1	0.1937203847727001	1.0218040943145752


In [8]:
ten_min_err_params = sorted(iter_alpha_err_time, key=lambda row: row[2])[:11]

print('iteration\talpha\terror\t\t\ttime')
for row in ten_min_err_params:
    print(f'{row[0]}\t\t{row[1]}\t{row[2]}\t{row[3]}')

iteration	alpha	error			time
19900		0.1	0.1937203847727001	1.0218040943145752
19800		0.1	0.19400727316663863	1.2761449813842773
19700		0.1	0.19429560076636448	1.1364946365356445
19600		0.1	0.19458537971942252	1.0699796676635742
19500		0.1	0.19487662234237701	1.179612398147583
19400		0.1	0.1951693411245076	1.1539998054504395
19300		0.1	0.1954635487316123	0.9777884483337402
19200		0.1	0.19575925800992838	1.0015418529510498
19100		0.1	0.19605648199016756	0.9711663722991943
19000		0.1	0.19635523389167628	1.0290813446044922
18900		0.1	0.19665552712672207	1.0574977397918701


In [9]:
# Инициализация весов, дающих минимальную ошибку
iterations, alpha, _, __ = ten_min_err_params[0]
W, err = eval_model(X_st, y, iterations, alpha)

print(W)

[-15.38680533  -1.7778509   -3.81676861  12.88454344]


### 3. Создайте функцию calc_pred_proba, возвращающую предсказанную вероятность класса 1 (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred_proba).

In [10]:
def calc_pred_proba(W, X):

    m = X.shape[1]    
    W = W.reshape(X.shape[0], 1)
    
    y_pred_proba = sigmoid(np.dot(W.T, X))[0]
    
    return y_pred_proba

In [11]:
y_pred_proba = [round(p, 2) for p in calc_pred_proba(W, X_st)]

y_pred_proba

[0.37, 0.11, 1.0, 0.0, 0.79, 0.25, 1.0, 0.07, 0.46, 1.0]

### 4. Создайте функцию calc_pred, возвращающую предсказанный класс (на вход подаются W, который уже посчитан функцией eval_model и X, на выходе - массив y_pred).

In [12]:
def calc_pred(W, X, threshold):
    
    y_pred_proba = calc_pred_proba(W, X)
    
    y_pred = [int(p >= threshold) for p in y_pred_proba]
    
    return y_pred

In [13]:
# Определение предсказанных значений
y_pred = calc_pred(W, X_st, 0.5)

### 5. Посчитайте Accuracy, матрицу ошибок, точность и полноту, а также F1 score.

In [14]:
m = len(y_pred)
indexes = range(0, m)

# Accuracy
accuracy = sum(y_pred[i] == y[i] for i in indexes) / m

# Матрица ошибок
TP = sum(y_pred[i] == 1 and y[i] == 1 for i in indexes)
FP = sum(y_pred[i] == 1 and y[i] == 0 for i in indexes)

FN = sum(y_pred[i] == 0 and y[i] == 1 for i in indexes)
TN = sum(y_pred[i] == 0 and y[i] == 0 for i in indexes)

# Точность и полнота
precision = TP / (TP + FP)
recall = TP / (TP + FN)

# F-мера
F_metric = lambda p, r, b: (1 + b ** 2) * (precision * recall) / (b ** 2 * precision + recall)
F1 = F_metric(precision, recall, 1)

In [15]:
print(f'''

y:\t{y}
y_pred:\t{y_pred}

Матрица ошибок
\t\ty = +1\t\ty = -1
a(x) = +1\t{TP}\t\t{FP}
a(x) = -1\t{FN}\t\t{TN}

Accuracy = {accuracy}
Точность = {precision}
Полнота = {recall}
F-мера с весом 1 = {F1:.2f}
''')



y:	[0. 0. 1. 0. 1. 0. 1. 0. 1. 1.]
y_pred:	[0, 0, 1, 0, 1, 0, 1, 0, 0, 1]

Матрица ошибок
		y = +1		y = -1
a(x) = +1	4		0
a(x) = -1	1		5

Accuracy = 0.9
Точность = 1.0
Полнота = 0.8
F-мера с весом 1 = 0.89



### 6. Могла ли модель переобучиться? Почему?

#### Модель переобучилась из-за большого последнего веса. Это случилось по причине малого количества данных.